![msg](msg.jpg)

In [15]:
#Imports 
import astropy.units as u
import astropy.constants as const
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns 

from tqdm import tqdm

from DiscEvolution.grid import Grid
from DiscEvolution.star import SimpleStar
from DiscEvolution.eos import LocallyIsothermalEOS  
from DiscEvolution.disc import AccretionDisc
from DiscEvolution.viscous_evolution import ViscousEvolutionFV
from DiscEvolution.driver import PlanetDiscDriver
from DiscEvolution.io import Event_Controller

In [16]:
# Physical parameters
#Assumed
Mstar = 1.0 * u.solMass #Default 1 solmass 
Rstar = 1.0 * u.solRad #Default 1 solRad
mu = 2.33  # Mean molecular weight, assumed

#Given
M_disk_0 = 0.05 * u.solMass  #Initial Disk Mass, default is 0.05 solmass
gamma = 1.0
Rc = 1.0 * u.AU #Default 1AU
#Mdot_0 = 
t_final = 1 * u.Myr 

# LBP surface density profile with gamma = 1: Sigma(r) = (M_disk / (2*pi*Rc*r)) * exp(-r/Rc)
def Sigma_LBP_1(r) -> float:
    val = (M_disk_0)/(2* np.pi * Rc * r) * np.exp((-1*r)/(Rc))
    if (val.cgs.unit != (u.g / (u.cm)**2)):
        print("Sigma_1 error")
        return 0
    return val.cgs

def Temperature(r) -> float:
    T_0 = 150 * u.K 
    val  = T_0 * (r / (1.0 * u.AU))**(-1/2)
    if val.cgs.unit != u.K:
        print('Temperature error')
        return 0
    return val.cgs

#Taken from A2
def Omega(r) -> float:
    val = np.sqrt((const.G*Mstar)/(r**3))
    if (val.cgs.unit != (1 / u.s)): 
        print("Omega error")
        return 0
    return val.cgs

#Taken from A2
def c_s(r) -> float:
    val = np.sqrt((const.k_B*Temperature(r))/(mu*const.m_p))
    if (val.cgs.unit != (u.cm / u.s)): 
        print("c_s error")
        return 0
    return val.cgs

def H(r) -> float: 
    val = c_s(r)/Omega(r)
    if val.cgs.unit != (u.cm):
        print("H error")
        return 0
    return val.cgs

In [17]:
#Given Mdisk, Mdot(0), Rc -> Calculate alpha and Sigma0
M_disk_0 = 0.05 * u.solMass
Mdot_0 = 1E-7 * u.solMass / u.yr  #Armitage p.21
Rc = 1.0 * u.AU

#Armitage eq 74
#nu*Sigma = Mdot/(3pi) * (1 - sqrt(r_star/r))
##alpha * c_s * H * Sigma = (Mdot / 3*pi) * (1 - sqrt(r_star/r))
###alpha = (Mdot / 3*pi) / (c_s * H * Sigma) * (1 - sqrt(r_star/r))

def alpha(r) -> float:
    # Convert all inputs to CGS values (no units)
    Mdot_cgs = Mdot_0.to(u.g / u.s).value  # g/s
    cs_val = c_s(r).value                   # cm/s
    H_val = H(r).value                      # cm
    Sigma_val = Sigma_LBP_1(r).value       # g/cm²
    Rstar_val = Rstar.to(u.cm).value       # cm
    r_val = r.to(u.cm).value               # cm
    
    # Calculate dimensionless alpha
    alpha = (Mdot_cgs / (3*np.pi)) * (1/(cs_val * H_val * Sigma_val)) * (1 - np.sqrt(Rstar_val/r_val))
    
    return alpha  # Pure float, dimensionless

r_ref = Rc #Not Rstar
alpha_val = alpha(r_ref)

Sigma_0 = Sigma_LBP_1(r_ref)

print(f"Accretion rate: {Mdot_0:.3e}")
print(f"At radius: {r_ref.to(u.AU):.3f}")
print(f"\nCalculated alpha: {alpha_val:.3e}")
print(f"Sigma_0: {Sigma_0.cgs:.3e}")


Accretion rate: 1.000e-07 solMass / yr
At radius: 1.000 AU

Calculated alpha: 8.973e-04
Sigma_0: 2.601e+04 g / cm2


In [18]:
# 1) Create grid
R_in = 0.1  # AU
R_out = 10.0  # AU
Ncells = 100

grid = Grid(R_in, R_out, Ncells, spacing='log')

# 2) Create star
star = SimpleStar(M=1.0)

In [19]:
# Custom EOS class with fixed LBP temperature profile
class FixedLBP_EOS(LocallyIsothermalEOS):
    """LocallyIsothermalEOS with fixed LBP temperature profile"""
    
    def __init__(self, star, cs_arr, H_arr, alpha_t, mu=2.4, R_arr=None):
        """
        args:
            star: Star object
            cs_arr: Sound speed array (custom profile) [cm/s]
            H_arr: Scale height array (custom profile) [cm]
            alpha_t: Turbulent alpha (constant)
            mu: Mean molecular weight
            R_arr: Radial grid (for interpolation)
        """
        # Initialize parent class minimally
        self._gamma = 1.0
        self._mu = mu
        self._alpha_t = alpha_t
        
        # Store custom profiles
        self._cs_custom = cs_arr
        self._H_custom = H_arr
        self._R_custom = R_arr if R_arr is not None else np.arange(len(cs_arr))
        
        # Dummy values (won't be used, but needed for parent interface)
        self._h0 = 0.1
        self._cs0 = 0.1
        self._q = 0
        self._H0 = 0.1
        self._T0 = 150
    
    def _f_cs(self, R):
        """Return sound speed from custom profile via interpolation"""
        return np.interp(R, self._R_custom, self._cs_custom)
    
    def _f_H(self, R):
        """Return scale height from custom profile via interpolation"""
        return np.interp(R, self._R_custom, self._H_custom)
    
    def _f_nu(self, R):
        """Viscosity = alpha_t * cs * H from custom profiles"""
        return self._alpha_t * self._f_cs(R) * self._f_H(R)
    
    def _f_alpha(self, R):
        """Constant alpha everywhere"""
        return self._alpha_t * np.ones_like(np.atleast_1d(R))
    
    @property
    def T(self):
        """Temperature from sound speed: T = cs^2 * mu / (k_B / m_p)"""
        from DiscEvolution.constants import GasConst
        return GasConst * self.cs**2 / self._mu

In [20]:
# 3) Create custom profiles on the grid
T_custom = np.array([Temperature(r * u.AU).value for r in grid.Rc])
cs_custom = np.array([c_s(r * u.AU).value for r in grid.Rc])
H_custom = np.array([H(r * u.AU).value for r in grid.Rc])

print(f"Custom profiles created:")
print(f"  T at 1 AU: {T_custom[np.argmin(np.abs(grid.Rc - 1.0))]:.1f} K")
print(f"  cs at 1 AU: {cs_custom[np.argmin(np.abs(grid.Rc - 1.0))]:.2e} cm/s")
print(f"  H at 1 AU: {H_custom[np.argmin(np.abs(grid.Rc - 1.0))]:.2e} cm")

# 4) Create custom EOS with fixed profiles
eos = FixedLBP_EOS(star, cs_custom, H_custom, alpha_t=alpha_val, R_arr=grid.Rc, mu=mu)
eos.set_grid(grid)

# 5) Create surface density from LBP profile
Sigma_initial = np.array([Sigma_LBP_1(r * u.AU).value for r in grid.Rc])

# 6) Create disc
disc = AccretionDisc(grid, star, eos, Sigma=Sigma_initial)
disc._planetesimal = False

# Verify setup
print(f"\nDisc setup complete:")
print(f"  Grid: {disc.Ncells} cells from {grid.Rc[0]:.2f} to {grid.Rc[-1]:.2f} AU")
print(f"  Disc mass: {disc.Mtot()/const.M_sun.to(u.g).value:.3e} M_sun")
print(f"  Alpha (constant): {alpha_val:.3e}")
print(f"  Nu at 1 AU: {disc.nu[np.argmin(np.abs(disc.R - 1.0))]:.2e} cm²/s")
print(f"  Nu at 30 AU: {disc.nu[-1]:.2e} cm²/s")
print(f"  Max nu: {disc.nu.max():.2e} cm²/s")

Custom profiles created:
  T at 1 AU: 151.7 K
  cs at 1 AU: 7.33e+04 cm/s
  H at 1 AU: 3.56e+11 cm

Disc setup complete:
  Grid: 100 cells from 0.10 to 9.77 AU
  Disc mass: 4.526e-02 M_sun
  Alpha (constant): 8.973e-04
  Nu at 1 AU: 2.34e+13 cm²/s
  Nu at 30 AU: 2.34e+14 cm²/s
  Max nu: 2.34e+14 cm²/s


In [21]:
# 7) Set up viscous evolution
visc_evol = ViscousEvolutionFV(tol=0.5, boundary='power_law', in_bound='Mdot')

# Create driver
driver = PlanetDiscDriver(disc, gas=visc_evol, t0=0.0)

# Check initial timestep
dt_max_initial = visc_evol.max_timestep(disc)
print(f"Initial timestep diagnostic:")
print(f"  dt_max: {dt_max_initial:.3e} seconds")
print(f"  dt_max: {dt_max_initial * (1*u.s).to(u.yr).value:.3e} years")
print(f"  dt_max: {dt_max_initial * (1*u.s).to(u.Myr).value:.3e} Myr")
print(f"\n  Viscosity range: {disc.nu.min():.2e} to {disc.nu.max():.2e} cm²/s")
print(f"  Grid spacing (inner): {(disc.R[1] - disc.R[0]):.3e} AU")

# Estimate number of steps
t_final_sec = t_final.to(u.s).value
n_steps_estimate = t_final_sec / dt_max_initial
print(f"\n  Estimated steps to {t_final.to(u.Myr).value} Myr: {n_steps_estimate:.1e}")

Initial timestep diagnostic:
  dt_max: 7.551e-19 seconds
  dt_max: 2.393e-26 years
  dt_max: 2.393e-32 Myr

  Viscosity range: 2.45e+12 to 2.34e+14 cm²/s
  Grid spacing (inner): 4.823e-03 AU

  Estimated steps to 1.0 Myr: 4.2e+31
